In [38]:
import pandas as pd
import numpy as np
import seaborn  as sns
import matplotlib.pyplot as plt
import math
import time as t
from pylab import rcParams
%matplotlib inline
%config IPCompleter.greedy=True

In [39]:
def readData():
    """
    load data from the file path
    factory function
    """
    #data_long = pd.read_excel("/Users/dilruba_p/Desktop/dataset_576_long.xlsx") #Dilruba mac
    data_long = pd.read_excel("/Users/haominshi/Desktop/al_data/dataset_576_long.xlsx") #Haomin mac
    #data_long = pd.read_excel("/home/haomin/Desktop/bio-data/dataset_576_long.xlsx") # thinkpad ubuntu
    df = data_long.copy()
    print("Finish Reading Data /dataset_576_long.xlsx")
    return df

def removeNoise(df):
    """
    Argument = df
    return = df
    
    drop some noise of the that might pollute the result
    """
    #medications with analgesic_rx
    #df = df[df.analgesic_rx !=1]
    #removing people who have a suspected of having head injuries in past visits 
    #df = df[df.headinjrloc_cum !=1]
    #removing people who have a suspected of having cancer in past visits 
    #df = df[df.cancer_cum !=1]
    #removing people who have a suspected of having diabetes in past visits 
    df = df[df.dm_cum !=1]
    print(df.shape)
    return df

def groupbyid(df):
    """
    arg = df
    return = list of df
    
    convert the df to a list of df that is grouped by the patient id
    return a list of df
    """
    # convert the values from the projid to a list
    projid_all = df.projid.unique()
    projid_list = projid_all.tolist() # a unique list of ID  acquired 

    patient_dfbyid = []
    start_df_row = 0 # selecting the first row 
    end_df_row   = 0 # selecting the last row
    flag_start   = 0 # start index for df spliting based on unique ID
    df = df.reset_index(drop = True) # df reindexed

    for p_id in projid_all: # loop through the unique id list
        for df_index in range(flag_start, df.shape[0]): 
            if df.iloc[df_index]['projid'] == p_id: end_df_row += 1
            elif df.iloc[df_index]['projid'] != p_id: break
        patient_dfbyid.append(df[start_df_row: end_df_row])
        start_df_row = end_df_row
        flag_start = end_df_row

    return patient_dfbyid # a list of df, that each element of the list is one patient

def patient_diagnosis(list_of_patient):
    """
    arg = list
    return list
    
    This function split the patient list to 2 sub list, each of them contain a whole DF for that patient
    The difference is that one list is for patient that is diagonsed in future, the other is not
    it is splif via the dcfdx value, dcfdx >= 3 means diagnosed
    """

    patient_dfbyid_diagnosed = []      #  diagnosed with alzheimer
    patient_dfbyid_not_diagnosed = []  # not diagnalzheimer
    diagonosed = False

    for patient in list_of_patient:
        dcfdx_list = patient['dcfdx'].tolist()
        for dcfdx_element in dcfdx_list:
            if dcfdx_element >= 3: # >= 4 means diagnoised
                diagonosed = True
                break
        if (diagonosed) == True:
            patient_dfbyid_diagnosed.append(patient)
            diagonosed= False
        else: patient_dfbyid_not_diagnosed.append(patient)
    
    return patient_dfbyid_diagnosed, patient_dfbyid_not_diagnosed

def appending_initial_df_to_full_df(list_of_df):
    """
    argument = list
    return = df
    
    This list takes in a list of dataframe and appending the first df to achieve concat, it returns
    the first df at the end
    """
    list_of_df_copy = list_of_df[:]
    counter = 0
    for df in list_of_df_copy[1:]:
        counter+=1
        print("appending{}".format(counter))
        list_of_df_copy[0] = list_of_df_copy[0].append(df, ignore_index=True)
    return list_of_df_copy[0]

def imputemean_per_id(list_of_df_temp,focus = "crn"):
    for df in list_of_df_temp:
        M = df[focus].mean()
        df[focus].fillna(M,inplace=True)
    return list_of_df_temp

def some_cleaning(listOfDf):
    """
    with special focus on certain columns, if any of these columns in the data is nan
    the row with that column is dropped, column comes form the string in the method
    
    argumnet = list
    return = list
    """
    #feature_str = \
    #"chlstrl gfr_mdrs glucose hdlchlstrl ldlchlstrl cogn_global cts_fruits cts_idea cts_sdmt iadlsum katzsum rosbscl rosbsum phys5itemsum"
    #label = "dcfdx"
    
    feature_str = \
    "gfr_mdrs cogn_global cts_fruits cts_idea cts_sdmt iadlsum rosbsum"
    label = "dcfdx"
    
    feature_item_list = feature_str.split(" ")
    # only pick rows that have these values
    test_list_patient = patient_visit_list[:]
    for df in test_list_patient:
        for feature in feature_item_list:
            df = df[np.isfinite(df[feature])]
    return test_list_patient

def fillNaNs(list_of_df):
    for df in list_of_df:
        pass
    


In [40]:
raw_dataframe = readData() # load excel to df
print("Done 1")
print(raw_dataframe.shape)
noise_removed_dataframe = removeNoise(raw_dataframe) # remove some noise
print("Done 2")
patient_visit_list = groupbyid(noise_removed_dataframe) # put df to a list grouped by patient id
print("Done 3")
patient_diagnosed, patient_not_diagnosed = patient_diagnosis(patient_visit_list)
print("{} amount people diagnosed, {} amount of people not diagnosed"\
      .format(len(patient_diagnosed), len(patient_not_diagnosed)))
print("{} total patient amount".format(len(patient_visit_list)))
print("Cell finished")

print(noise_removed_dataframe.shape)

Finish Reading Data /dataset_576_long.xlsx
Done 1
(25570, 203)
(21254, 203)
Done 2
Done 3
841 amount people diagnosed, 2048 amount of people not diagnosed
2889 total patient amount
Cell finished
(21254, 203)


In [41]:
list_of_columns = ['chlstrl','crn','gfr_mdrs','glucose','hba1c','hdlchlstrl', 'hdlratio','hemacrit','hemoglbn',
'ldlchlstrl', 'mch', 'mchc', 'mcv', 'platelet','rbc', 'dcfdx', 'projid', 'age_at_visit'] 
# these are the columns that we are interesed in, However, right now i dont have a good idea about what value I
# should be use to replace the NaN

__One of the approach we have is to replace the NaNs with local mean of that column value based on that patient's visit. Personally, I dont think that is a good idea. Certainly, if a patient have dcfdx span from 1 to 4, the mean then lost its meanings.__

Check the diabieties, how?
1. Person is in first visit, and he/she does not have AD and D
2. Person is in second visit, current they have dcfdx == 4, we want to see next visit if they have D
3. Person is in third or more visit, with AD already diagnosed check if they have D

Some special trails:
1. I tried not to drop anyone who is diagnosed with Ob but this seems not really affect the outcome, even tho the original sample size is much larger

In [42]:
# some experiments
print("{} original len".format(len(patient_visit_list)))
def check_dcfdx_dP_clean_1(patient_list_id):
    """
    check if there is a relation between poeple who have AD and D
    bmi 25, 30 this is overweight
    bmi 30, 40 this is ob
    bmi 40 + this is very ob
    """
    return_list_of_patent = []
    for patient in patient_list_id:
        if patient.iloc[0]['dcfdx'] == 1 and patient.iloc[0]['bmi'] < 25:
            return_list_of_patent.append(patient)
            continue # skip to next patient
    return return_list_of_patent

def check_dcfdx_dp_clean_2(patient_list_id):
    return_list_of_patent = []
    for patient in patient_list_id:
        for row_index in range(patient.shape[0]):
            if patient.iloc[row_index]['dcfdx'] >= 4 and patient.iloc[row_index]['bmi'] >= 25:
                return_list_of_patent.append(patient)
                break # no need to scan if found
    return return_list_of_patent

def check_dcfdx_dp_clean_special(patient_list_id):
    return_list_of_patent = []
    for patient in patient_list_id:
        for row_index in range(patient.shape[0]):
            if patient.shape[0] < 3:
                break # dont care about patient has less visit than 3
            if patient.iloc[row_index]['dcfdx'] >= 4 and patient.iloc[row_index]['bmi'] >= 25:
                return_list_of_patent.append(patient)
                break # no need to scan if found
    return return_list_of_patent


test_list_stage1 = check_dcfdx_dP_clean_1(patient_visit_list)
print("{} stage 1 clean".format(len(test_list_stage1)))
test_list_stage2 = check_dcfdx_dp_clean_2(test_list_stage1)
print("{} stage 2 clean".format(len(test_list_stage2)))
test_list_stage_special = check_dcfdx_dp_clean_special(patient_visit_list)
print("{} stage 3 clean".format(len(test_list_stage_special)))
            

2889 original len
702 stage 1 clean
33 stage 2 clean
349 stage 3 clean


In [43]:
focus_patient_list = some_cleaning(patient_visit_list)

In [ ]:
# Calculate counter and the percentage of ppl that is sick
counter = 0

for patient in focus_patient_list:
    for row_number in range(patient.shape[0]):
        if patient.iloc[row_number]['dcfdx'] >= 4:
            counter+=1
            break
                
print("there are total {} patient, within that there are {} people was dcfdx >= 4 ignore previous record, \n that is {} %"\
     .format(len(focus_patient_list), counter, counter/len(focus_patient_list) * 100))

counter = 0
for patient in focus_patient_list:
    for row_number in range(patient.shape[0]):
        if patient.iloc[0]['dcfdx'] == 2:
            if patient.iloc[row_number]['dcfdx'] >= 4:
                counter+=1
                break
print("there are total {} patient, within that there are {} people was dcfdx <= 3 to dcfdx >= 4, \n that is {} %"\
     .format(len(focus_patient_list), counter, counter/len(focus_patient_list) * 100))

counter = 0
for patient in focus_patient_list:
    for row_number in range(patient.shape[0]):
        if patient.iloc[0]['dcfdx'] == 1:
            if patient.iloc[row_number]['dcfdx'] >= 4:
                counter+=1
                break
print("there are total {} patient, within that there are {} people was dcfdx == 1 to dcfdx >= 4, \n that is {} %"\
     .format(len(focus_patient_list), counter, counter/len(focus_patient_list) * 100))

# some explore

counter = 0
for patient in focus_patient_list:
    for row_number in range(patient.shape[0]):
        if patient.shape[0] > 2:
            if patient.iloc[0]['dcfdx'] <= 2 or patient.iloc[1]['dcfdx'] <= 2 or patient.iloc[2]['dcfdx'] <= 2:
                if patient.iloc[row_number]['dcfdx'] >= 4:
                    counter+=1
                    break
print("there are total {} patient, within that there are {} in explore function, \n that is {} %"\
     .format(len(focus_patient_list), counter, counter/len(focus_patient_list) * 100))



there are total 2889 patient, within that there are 818 people was dcfdx >= 4 ignore previous record, 
 that is 28.314295604015232 %
there are total 2889 patient, within that there are 283 people was dcfdx <= 3 to dcfdx >= 4, 
 that is 9.795777085496711 %
there are total 2889 patient, within that there are 355 people was dcfdx == 1 to dcfdx >= 4, 
 that is 12.287988923502942 %
